<div >
<img src = "../banner.jpg" />
</div>

<a href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202520/blob/main/Lecture13/Notebook_SpatialData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Data

In [ ]:
require("pacman")
p_load("tidyverse","sf","modeldata")

In [ ]:
data("ames", package = "modeldata")

In [ ]:
head(ames)

In [ ]:
dim(ames)

In [ ]:
class(ames)

![](figs/mercator.gif)

In [ ]:
#For speed I'm going to keep the ten neighborhoods with most observations
ames<-ames  %>% filter(Neighborhood %in%c("North_Ames", "College_Creek", "Old_Town", "Edwards", "Somerset", "Northridge_Heights", "Gilbert", "Sawyer", "Northwest_Ames", "Sawyer_West"))

ames_sf <- sf::st_as_sf(
  ames,
  # "coords" is in x/y order -- so longitude goes first!
  coords = c( "Longitude","Latitude"),
  remove=FALSE,
  # Set our coordinate reference system to EPSG:4326,
  # the standard WGS84 geodetic coordinate reference system
  crs = 4326
)

In [ ]:
#?st_as_sf

In [ ]:
class(ames_sf)

In [ ]:
head(ames_sf)

In [ ]:
#graficar con ggplot
ggplot() +
    geom_sf(data=ames_sf)+
    theme_bw()

In [ ]:
p_load("leaflet")

In [ ]:
map1<-leaflet()  %>% 
        addTiles()  %>% 
        addCircleMarkers(data=ames_sf)
map1

## Spatial Autocorrelation

This relationship may exhibit spatial autocorrelation across the city of Ames, and we can investigate it using any of the several methods provided by the spatialsample. 


In [ ]:
p_load("spatialsample")

### Spatial Blocks

For instance, the `spatial_block_cv()` function will perform [spatial blocking](https://doi.org/10.1111/ecog.02881) with your data:

In [ ]:
set.seed(123)
block_folds <- spatial_block_cv(ames_sf, v = 5)

Here, the seed ensures that the sampling results are reproducible. Then, `spatial_block_cv` divides the spatial dataset `ames_sf` into 5 folds for cross-validation, ensuring the training and testing sets are spatially separated. This prevents information from geographically close observations from leaking between folds. 

`Autoplot` will give us a clear visual of how the data was split into blocks, helping us understand the spatial structure of the validation scheme.

In [ ]:
p_load("purrr")

walk(block_folds$splits, function(x) print(autoplot(x)))

### Spatial LLOCV

If you already have a sense of what locations in your data are likely to be closely related, you can also use the `spatial_leave_location_out_cv()` function to perform [leave-location-out cross-validation](https://doi.org/10.1016/j.envsoft.2017.12.001). 

For instance, we can split the Ames data into folds based on neighborhoods using this function:

In [ ]:
set.seed(123)

location_folds <- 
  spatial_leave_location_out_cv(
    ames_sf,
    group = Neighborhood
  )

In [ ]:
p_load("purrr")

walk(location_folds$splits, function(x) print(autoplot(x)))

### Spatial Buffers

The `spatial_buffer_vfold_cv()` function will perform [spatially buffered cross-validation](https://onlinelibrary.wiley.com/doi/10.1111/geb.12161) with your data:

In [ ]:
set.seed(123)

buffer_folds <- spatial_buffer_vfold_cv(ames_sf, radius=200,buffer=200)

In [ ]:
autoplot(buffer_folds$splits[[3]]) + theme_bw()

 Here we use a 200-meter radius and a 100-meter buffer. Although the input data is in degrees (EPSG:4326), `spatial_buffer_vfold_cv()` appears to perform a **sanity check**: since 200 degrees would represent an enormous distance (over half the Earth's circumference, which is approximately 40,075 km), the function internally treats the values as if they are in **meters** rather than blindly applying them as degrees. This behavior likely prevents nonsensical results when users provide realistic buffer sizes but forget to project the data. 


Nonetheless, it's **best practice** to explicitly transform your data to a projected CRS in meters (e.g., UTM) to avoid relying on implicit assumptions and ensure spatial distances are handled consistently.



In [ ]:
ames_sf2 <- st_transform(ames_sf, crs = 26915)  # UTM zone 15N, in meters   https://spatialreference.org/ref/epsg/26915/

set.seed(123)
buffer_folds2 <- spatial_buffer_vfold_cv(ames_sf2, radius = 200, buffer = 200)

In [ ]:
autoplot(buffer_folds2$splits[[3]]) + theme_bw()

## Full implementation with Elastic Net

\begin{align}
min_{\beta} EN(\beta) &= \sum_{i=1}^n (y_i-\beta_0 - \sum_{j=1}^p x_{ij}\beta_j)^2  + \lambda\left(\alpha \sum_{j=1}^p |\beta_j| + \frac{(1-\alpha)}{2} \sum_{j=1}^p (\beta_j)^2\right)
\end{align}

In [ ]:
folds<-list()

for(i in 1:10){
  folds[[i]]<- location_folds$splits[[i]]$in_id
}


In [ ]:
head(folds[[2]])

In [ ]:
folds[[2]][!(folds[[2]]%in%folds[[1]])]

In [ ]:
p_load("caret")

fitControl<-trainControl(method ="cv",
                         index=folds)



In [ ]:
EN<-train(log(Sale_Price) ~ Gr_Liv_Area  +  Bldg_Type ,
             data=ames_sf,
             method = 'glmnet', 
             trControl = fitControl,
             tuneGrid = expand.grid(alpha =seq(0,1,length.out = 20),
                                    lambda = seq(0.001,0.2,length.out = 50))
              ) 

In [ ]:
EN

In [ ]:
EN$bestTune

In [ ]:
round(EN$results$RMSE[which.min(EN$results$lambda)],4)

In [ ]:
set.seed(123)

fitControl2<-trainControl(method ="cv",
                         number=5)

EN2<-train(log(Sale_Price) ~ Gr_Liv_Area  +  Bldg_Type ,
             data=ames_sf,
             method = 'glmnet', 
             trControl = fitControl2,
             tuneGrid = expand.grid(alpha =seq(0,1,length.out = 20),
                                    lambda = seq(0.001,0.2,length.out = 50))
              ) 



In [ ]:
round(EN2$results$RMSE[which.min(EN2$results$lambda)],4)

## Example Problem Set

In [ ]:
test<- ames_sf  %>% filter(Neighborhood=="North_Ames")

train<-ames_sf  %>% filter(Neighborhood!="North_Ames")

In [ ]:
set.seed(123)

location_folds_train <- 
  spatial_leave_location_out_cv(
    train,
    group = Neighborhood
  )



In [ ]:
autoplot(location_folds_train)

In [ ]:
folds_train<-list()
for(i in 1:length(location_folds_train$splits)){
  folds_train[[i]]<- location_folds_train$splits[[i]]$in_id
}


In [ ]:
fitControl_tp_random<-trainControl(method ="cv",
                         number=5)

fitControl_spatial<-trainControl(method ="cv",
                         index=folds_train)

In [ ]:
set.seed(123)

EN_tp_random<-train(log(Sale_Price) ~ Gr_Liv_Area:Bldg_Type ,
             data=train,
             method = 'glmnet', 
             trControl = fitControl_tp_random,
             metric="MAE",
             tuneGrid = expand.grid(alpha =seq(0,1,length.out = 10),
                                    lambda = seq(0.001,0.2,length.out = 10))
              ) 

In [ ]:
set.seed(123)

EN_tp_spatial<-train(log(Sale_Price) ~ Gr_Liv_Area:Bldg_Type ,
             data=train,
             method = 'glmnet', 
             trControl = fitControl_spatial,
             metric="MAE",
             tuneGrid = expand.grid(alpha =seq(0,1,length.out = 10),
                                    lambda = seq(0.001,0.2,length.out = 10))
              ) 

In [ ]:
#EN_tp_random

In [ ]:
#EN_tp$bestTune

In [ ]:
test$log_price_hat_random<-predict(EN_tp_random,newdata = test)

In [ ]:
head(test  %>% select(Sale_Price,log_price_hat_random)  %>% st_drop_geometry())

In [ ]:
test$log_price_hat_spatial<-predict(EN_tp_spatial,newdata = test)

In [ ]:
test<- test  %>% mutate(price_hat_random=exp(log_price_hat_random),price_hat_spatial=exp(log_price_hat_spatial))

In [ ]:
head(test)

#### What is Kaggle's score?

In [ ]:
#MAE
mean(abs(test$Sale_Price-test$price_hat_random))

In [ ]:
mean(abs(test$Sale_Price-test$price_hat_spatial))

In [ ]:
#MAE
mean(abs(test$Sale_Price-floor(test$price_hat_spatial)))